# Regrid Datasets

Now that we have downloaded our data sets in the [data compilation notebook](0_compile_datasets.ipynb), we will regrid the ET datasets using the conservative method within [`xarray-regrid`](https://github.com/xarray-contrib/xarray-regrid). (See this [notebook](https://github.com/xarray-contrib/xarray-regrid/blob/main/docs/notebooks/benchmarks/benchmarking_conservative.ipynb) for an example.) The `conservative` method is chosen as it is an area weighted average, which is optimal for decreasing resolution.

In [ ]:
import hvplot.xarray
import xarray as xr
import xarray_regrid
import panel as pn

Let's read in our data sets that we compiled in the [data compilation notebook](0_compile_datasets.ipynb).

In [ ]:
ssebop = xr.open_dataset(
    "../Data/ssebop/ssebop_aet.nc",
    engine="netcdf4",
    chunks={"lon": -1, "lat": -1, "time": 2},
)
gleam = xr.open_dataset(
    "../Data/gleam/gleam_aet.nc",
    engine="netcdf4",
    chunks={"lat": -1, "lon": -1, "time": -1},
)
era5 = xr.open_dataset(
    "../Data/era5/era5_aet.nc",
    engine="netcdf4",
    chunks={"lon": -1, "lat": -1, "time": 219},
)
nldas = xr.open_dataset(
    "../Data/nldas/nldas_aet.nc",
    engine="netcdf4",
    chunks={"lat": -1, "lon": -1, "time": -1},
)
terra = xr.open_dataset(
    "../Data/terraclimate/terraclimate_aet.nc",
    chunks={"lat": -1, "lon": -1, "time": 30},
)
wbet = xr.open_dataset(
    "../Data/wbet/wbet_aet.nc",
    engine="netcdf4",
    chunks={"lat": -1, "lon": -1, "time": 2},
)

Either by looking at the data sets or from general knowledge of the data, GLEAM has the lowest resolution of the six data sets. So, we will decrease the resolution of the other datasets to match its resolution. To do this, we just need to extract the lat/long grid from the GLEAM data set to use as our target grid. We then regrid.

In [ ]:
ds_to_regrid = [ssebop, era5, nldas, terra, wbet]
ds_names = ["ssebop", "era5", "nldas", "terraclimate", "wbet"]

target_grid = gleam[["lat", "lon"]]

for ds, name in zip(ds_to_regrid, ds_names):
    ds_regridded = ds.regrid.conservative(
        target_grid, latitude_coord="lat", skipna=True,
        nan_threshold=0, output_chunks={"lat": -1, "lon": -1, "time": -1}
    )
    ds_regridded.to_netcdf(
        path="../Data/" + name + "/" + name + "_aet_regridded.nc",
        format="NETCDF4",
        engine="netcdf4",
    )

Now that we have regridded, let's see how the regridded data sets compare with the GLEAM data set. We will read in the regridded data and plot the maps.

In [ ]:
ssebop_regrid = xr.open_dataset(
    "../Data/ssebop/ssebop_aet_regridded.nc", engine="netcdf4"
)
era5_regrid = xr.open_dataset("../Data/era5/era5_aet_regridded.nc", engine="netcdf4")
nldas_regrid = xr.open_dataset("../Data/nldas/nldas_aet_regridded.nc", engine="netcdf4")
terra_regrid = xr.open_dataset(
    "../Data/terraclimate/terraclimate_aet_regridded.nc", engine="netcdf4"
)
wbet_regrid = xr.open_dataset("../Data/wbet/wbet_aet_regridded.nc", engine="netcdf4")

In [ ]:
plt = (
    ssebop_regrid.hvplot(
        groupby="time", geo=True, coastline=True, title="SSEBop Regridded"
    ).opts(frame_width=400)
    + gleam.hvplot(
        groupby="time", geo=True, coastline=True, title="GLEAM Original"
    ).opts(frame_width=400)
    + era5_regrid.hvplot(
        groupby="time", geo=True, coastline=True, title="ERA5 Regridded"
    ).opts(frame_width=400)
    + nldas_regrid.hvplot(
        groupby="time", geo=True, coastline=True, title="NLDAS Regridded"
    ).opts(frame_width=400)
    + terra_regrid.hvplot(
        groupby="time", geo=True, coastline=True, title="TerraClimate Regridded"
    ).opts(frame_width=400)
    + wbet_regrid.hvplot(
        groupby="time", geo=True, coastline=True, title="WBET Regridded"
    ).opts(frame_width=400)
)

pn.panel(plt.cols(2), widget_location="top")